Phase 0 — Orientation & Tooling (Week 1)

* **Local dev environment ready**
    Repo folder: /Software_development/Python/projects/wallet_analytics
* **API key obtained**
    Alchemy API key: .env
* **Git repo initialized**
    https://github.com/Dev-Uchiha/wallet_analytics.git


To start, i created a repository folder on my laptop called wallet_analytics where everything related to the project can be stored.
I then installed a python package manager called "uv". A package manager is a python (programming language) toolkit which allows the user to install all the needed tools (packages of code) to create a project. For e.g to do maths using python, i would need to install "numpy" which will already have all the code needed to do calculations.
I chose uv specifically because it is much faster at downloading packages than the standard package manager "pip" and it is also what we use at work.
I then used uv to create a virtual environment inside the wallet_analytics folder. This virtual environment is the place where all the installed tools(code) related to the projects will reside. Each project should have its own virtual environment for code cleanliness.

I then created an account with Alchemy to get their API key (An API is a way for one software to speak to another). This key allows me to get data about Ethereum wallets from the Alchemy website.

Lastly, I installed "git", which allows me to keep track of the different changes i make to my code. I also connected it to "github" so that i can store my code on the github website.

Phase 1 — Raw On-Chain Data Ingestion (Weeks 2–3)
* Normal transactions
* ERC-20 token transfers

**Deliverable**

* Script that pulls data for 1 wallet
* Raw tables saved locally
* Re-runnable without manual edits


I then created various functions (blocks of code that have a spefic purpose), to return the last 10 transactions from vitalik buterins eth wallet, displaying the transaction hash (a unique transaction id), eth amount and data. This can be verified by putting his wallet address in an eth blockchain explorer (a website which allows you to track wallet data) to see if the last 10 transactions are the same. 

https://etherscan.io/address/0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045

The next step was to bring back all the relevant wallet data and save them locally as .csv files. 
This included:

**transactions**
tx_hash
block_number
timestamp
from_address
to_address
value_eth
gas_used
gas_price
status

**token_transfers**
tx_hash
token_symbol
token_contract
from_address
to_address
value
timestamp

The differences between transactions and token transfers are:
    - a transaction moves a value of eth (the base asset) from one wallet to another (e.g sending someone 5 eth). It initiates an execution and shows what was sent and who sent it.
    - a token transfer changes the ownership of a token from one wallet to another (e.g sending someone an nft). It shows what did the contract say happened. 
    - all token transfers happen within transactions, but a token transfer isnt needed for a transaction
    - transactions are recorded in the transaction object() but token transfers are recorded in the logs (a description of what happened in the contract)


## Phase 2 — Wallet-Level Analytics (Weeks 4–5)


### Derived Metrics (Core)

Per wallet:

* Total ETH received / sent
* Net ETH balance (from txs, not live balance)
* Gas spent (ETH)
* Transaction count
* Active days
* First / last activity

Token-level:

* Tokens interacted with
* Net token inflow/outflow
* Frequency by token

### Behavioral Metrics (Differentiator)

* Avg tx size
* Tx frequency over time
* % outgoing vs incoming
* Gas per transaction trend
* Dormant periods

### Output Tables

**wallet_summary**

```
wallet
first_tx_date
last_tx_date
tx_count
total_eth_sent
total_eth_received
net_eth
total_gas_spent
current_eth_balance
```

**wallet_activity**

```
wallet
period
tx_count
eth_sent
eth_received
gas_spent
```

**Deliverable**

* Clean analytical tables
* Documented metric definitions
* Deterministic transformations

In [ ]:
#Creating the virtual environment

cd users/name/project
uv venv .venv        # once
uv sync              # whenever deps change
source .venv/bin/activate

In [3]:
#Imports

# Return the last 10 transactions from an ethereum wallet
# Return the most relevant data headers

import requests
import pandas as pd
import time
import os
import json
from datetime import datetime
from dotenv import load_dotenv
import psycopg2
from psycopg2 import sql




In [4]:
#Config (API key, DB config, constants)

load_dotenv()

ALCHEMY_API_KEY = os.getenv("ALCHEMY_API_KEY")
ALCHEMY_BASE_URL = "https://eth-mainnet.g.alchemy.com/v2"
OUTPUT_DIR = "output"
CSV_COUNT = 100


DB_CONFIG = {
    "dbname": "wallet_analytics",
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": "localhost",
    "port": 5432
}



In [5]:
#DB connection and create_tables()
def get_db_connection():
    """Create and return a connection to the PostgreSQL database."""
    return psycopg2.connect(**DB_CONFIG)

# Test the connection
try:
    conn = get_db_connection()
    print("Connected to PostgreSQL successfully")
    conn.close()
except Exception as e:
    print(f"Connection failed: {e}")
    
def create_tables():
    """Create the wallet_summary and wallet_activity tables if they don't already exist."""
    conn = get_db_connection()
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS wallet_summary (
            wallet TEXT,
            first_tx_date DATE,
            last_tx_date DATE,
            tx_count BIGINT,
            total_eth_sent DOUBLE PRECISION,
            total_eth_received DOUBLE PRECISION,
            net_eth DOUBLE PRECISION,
            total_gas_spent DOUBLE PRECISION,
            current_eth_balance DOUBLE PRECISION
        );
    """)

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS wallet_activity (
            wallet TEXT,
            period DATE,
            tx_count BIGINT,
            eth_sent DOUBLE PRECISION,
            eth_received DOUBLE PRECISION,
            gas_spent DOUBLE PRECISION
        );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS token_transfers (
        wallet TEXT,
        tx_hash TEXT,
        token_symbol TEXT,
        token_contract TEXT,
        from_address TEXT,
        to_address TEXT,
        value DOUBLE PRECISION,
        timestamp TIMESTAMP
    );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS transactions (
        tx_hash TEXT,
        block_number BIGINT,
        timestamp TIMESTAMP,
        from_address TEXT,
        to_address TEXT,
        value_eth DOUBLE PRECISION,
        gas_used BIGINT,
        gas_price BIGINT,
        status TEXT,
        tx_type TEXT,
        wallet TEXT
    );
""")

    conn.commit()
    cursor.close()
    conn.close()
    print("Tables created successfully")

create_tables()


Connected to PostgreSQL successfully
Tables created successfully


In [6]:
#Alchemy fetch functions

def get_alchemy_json(method, params):
    """Make a request to Alchemy JSON-RPC."""
    url = f"{ALCHEMY_BASE_URL}/{ALCHEMY_API_KEY}"
    payload = {"id": 1, "jsonrpc": "2.0", "method": method, "params": params}

    r = requests.post(url, json=payload, headers={"Content-Type": "application/json"})
    r.raise_for_status()

    data = r.json()
    if "error" in data:
        raise Exception(f"Alchemy API error: {data['error']}")

    return data


def get_eth_balance(address):
    """Return ETH balance for an address (in ETH)."""
    data = get_alchemy_json("eth_getBalance", [address, "latest"])
    balance_wei = int(data["result"], 16)
    return balance_wei / 10**18



In [15]:
# Transactions/token transfers  functions

def _fetch_external_transfers(address, max_count):
    """Fetch up to max_count external (ETH) transfers for address; reuses get_alchemy_json from above."""
    base_params = {
        "fromBlock": "0x0",
        "toBlock": "latest",
        "category": ["external"],
        "maxCount": hex(max_count),
        "excludeZeroValue": False,
        "withMetadata": True,
        "order": "desc",
    }
    all_txs = []
    for key, val in [("toAddress", address), ("fromAddress", address)]:
        p = {**base_params, key: val}
        data = get_alchemy_json("alchemy_getAssetTransfers", [p])
        all_txs.extend(data.get("result", {}).get("transfers", []))
        time.sleep(0.1)
    df = pd.DataFrame(all_txs)
    if df.empty:
        return df
    if "hash" in df.columns:
        df = df.drop_duplicates(subset=["hash"], keep="first")
    if "blockNum" in df.columns:
        df["_bn"] = df["blockNum"].apply(
            lambda x: int(x, 16) if isinstance(x, str) and x.startswith("0x") else -1
        )
        df = df.sort_values("_bn", ascending=False).drop(columns=["_bn"])
    return df.head(max_count).reset_index(drop=True)


def get_transactions(address):
    """
    Return normal (external ETH) transactions as a DataFrame with schema:
    tx_hash, block_number, timestamp, from_address, to_address, value_eth, gas_used, gas_price, status.
    Displays last 10; contains last 100.
    """
    raw = _fetch_external_transfers(address, CSV_COUNT)
    if raw.empty:
        return pd.DataFrame(columns=[
            "tx_hash", "block_number", "timestamp", "from_address", "to_address",
            "value_eth", "gas_used", "gas_price", "status",
        ])

    rows = []
    for _, row in raw.iterrows():
        tx_hash = row.get("hash", "")
        block_hex = row.get("blockNum", "0x0")
        block_number = int(block_hex, 16) if isinstance(block_hex, str) and block_hex.startswith("0x") else 0
        meta = row.get("metadata") or {}
        timestamp = meta.get("blockTimestamp", "")
        from_addr = row.get("from", "")
        to_addr = row.get("to", "")
        val = row.get("value")
        if val is None:
            value_wei = 0
        elif isinstance(val, str) and str(val).startswith("0x"):
            value_wei = int(val, 16)
        else:
            value_wei = int(val) if val is not None else 0
        value_eth = value_wei / 10**18

        gas_used, gas_price, status = None, None, None
        try:
            tx_data = get_alchemy_json("eth_getTransactionByHash", [tx_hash])
            receipt = get_alchemy_json("eth_getTransactionReceipt", [tx_hash])
            time.sleep(0.05)
            if tx_data.get("result"):
                gas_price = int(tx_data["result"].get("gasPrice", "0x0"), 16)
            if receipt.get("result"):
                gas_used = int(receipt["result"].get("gasUsed", "0x0"), 16)
                status = "success" if int(receipt["result"].get("status", "0x0"), 16) == 1 else "failed"
        except Exception:
            pass

        rows.append({
            "tx_hash": tx_hash,
            "block_number": block_number,
            "timestamp": timestamp,
            "from_address": from_addr,
            "to_address": to_addr,
            "value_eth": value_eth,
            "gas_used": gas_used,
            "gas_price": gas_price,
            "status": status,
        })

    df = pd.DataFrame(rows)
    return df


def _fetch_token_transfers(address, max_count):
    """Fetch up to max_count ERC-20 token transfers for address."""
    base_params = {
        "fromBlock": "0x0",
        "toBlock": "latest",
        "category": ["erc20"],
        "maxCount": hex(max_count),
        "excludeZeroValue": False,
        "withMetadata": True,
        "order": "desc",
    }
    all_txs = []
    for key, val in [("toAddress", address), ("fromAddress", address)]:
        p = {**base_params, key: val}
        data = get_alchemy_json("alchemy_getAssetTransfers", [p])
        all_txs.extend(data.get("result", {}).get("transfers", []))
        time.sleep(0.1)
    df = pd.DataFrame(all_txs)
    if df.empty:
        return df
    if "hash" in df.columns:
        df = df.drop_duplicates(subset=["hash"], keep="first")
    if "blockNum" in df.columns:
        df["_bn"] = df["blockNum"].apply(
            lambda x: int(x, 16) if isinstance(x, str) and x.startswith("0x") else -1
        )
        df = df.sort_values("_bn", ascending=False).drop(columns=["_bn"])
    return df.head(max_count).reset_index(drop=True)


def get_token_transfers(address):
    """
    Return ERC-20 token transfers as a DataFrame with schema:
    tx_hash, token_symbol, token_contract, from_address, to_address, value, timestamp.
    Displays last 10; contains last 100.
    """
    raw = _fetch_token_transfers(address, CSV_COUNT)
    if raw.empty:
        return pd.DataFrame(columns=[
            "tx_hash", "token_symbol", "token_contract", "from_address", "to_address",
            "value", "timestamp",
        ])

    rows = []
    for _, row in raw.iterrows():
        meta = row.get("metadata") or {}
        raw_contract = row.get("rawContract") or {}
        token_contract = raw_contract.get("address", "") or ""
        # Alchemy may return 'asset' as symbol (e.g. "USDC"); fallback to contract or "N/A"
        token_symbol = row.get("asset") or row.get("symbol") or token_contract or "N/A"
        if isinstance(token_symbol, dict):
            token_symbol = token_symbol.get("symbol", "N/A") or "N/A"
        value_raw = row.get("value")
        if value_raw is None:
            value = None
        elif isinstance(value_raw, (int, float)):
            value = float(value_raw)
        else:
            try:
                value = int(str(value_raw), 16) if str(value_raw).startswith("0x") else float(value_raw)
            except (ValueError, TypeError):
                value = value_raw
        rows.append({
            "tx_hash": row.get("hash", ""),
            "token_symbol": str(token_symbol),
            "token_contract": token_contract,
            "from_address": row.get("from", ""),
            "to_address": row.get("to", ""),
            "value": value,
            "timestamp": meta.get("blockTimestamp", ""),
        })

    df = pd.DataFrame(rows)
    return df


In [8]:
# Analytics/metrics functions

def _parse_ts(ts):
    """Parse timestamp string to date for grouping. Handles ISO and common formats."""
    if ts is None or (isinstance(ts, float) and pd.isna(ts)):
        return None
    s = str(ts).strip()
    if not s:
        return None
    try:
        if "T" in s:
            return datetime.fromisoformat(s.replace("Z", "+00:00")).date()
        return datetime.strptime(s[:10], "%Y-%m-%d").date()
    except Exception:
        return None


def _gas_spent_eth(row):
    """Gas spent in ETH from gas_used and gas_price (wei)."""
    gu, gp = row.get("gas_used"), row.get("gas_price")
    if pd.isna(gu) or pd.isna(gp) or gu is None or gp is None:
        return 0.0
    try:
        return (int(gu) * int(gp)) / 1e18
    except (TypeError, ValueError):
        return 0.0


# --- Core metric functions (from transactions DataFrame for one wallet) ---

def total_eth_sent(tx_df, wallet_address):
    """Total ETH sent by wallet (outgoing value_eth where from_address == wallet)."""
    if tx_df.empty or "from_address" not in tx_df.columns:
        return 0.0
    mask = tx_df["from_address"].str.lower() == wallet_address.lower()
    return float(tx_df.loc[mask, "value_eth"].sum())


def total_eth_received(tx_df, wallet_address):
    """Total ETH received by wallet (incoming value_eth where to_address == wallet)."""
    if tx_df.empty or "to_address" not in tx_df.columns:
        return 0.0
    mask = tx_df["to_address"].str.lower() == wallet_address.lower()
    return float(tx_df.loc[mask, "value_eth"].sum())


def net_eth_from_txs(tx_df, wallet_address):
    """Net ETH from transactions (received - sent)."""
    return total_eth_received(tx_df, wallet_address) - total_eth_sent(tx_df, wallet_address)


def total_gas_spent_eth(tx_df):
    """Total gas spent in ETH across all transactions."""
    if tx_df.empty:
        return 0.0
    return float(tx_df.apply(_gas_spent_eth, axis=1).sum())


def first_last_activity_dates(tx_df):
    """Return (first_tx_date, last_tx_date) as date objects or (None, None)."""
    if tx_df.empty or "timestamp" not in tx_df.columns:
        return None, None
    dates = tx_df["timestamp"].apply(_parse_ts).dropna()
    if dates.empty:
        return None, None
    return dates.min(), dates.max()


def active_days_count(tx_df):
    """Number of distinct days with at least one transaction."""
    if tx_df.empty or "timestamp" not in tx_df.columns:
        return 0
    dates = tx_df["timestamp"].apply(_parse_ts).dropna()
    return int(dates.nunique())


# --- Tableau output tables ---

def get_wallet_summary_table(address):
    """
    Build wallet_summary table for one ETH wallet. Uses get_transactions() from previous cells.

    Columns: wallet, first_tx_date, last_tx_date, tx_count, total_eth_sent, total_eth_received, net_eth, total_gas_spent, current_eth_balance
    """
    try:
        get_transactions
    except NameError:
        raise NameError(
            "get_transactions is not defined. Run the Phase 1 cell "
            "'Transactions and token_transfers in project schema' first."
        )
    tx_df = get_transactions(address)
    wallet = address
    current_balance = get_eth_balance(address)

    if tx_df.empty:

        row = {
            "wallet": wallet,
            "first_tx_date": None,
            "last_tx_date": None,
            "tx_count": 0,
            "total_eth_sent": 0.0,
            "total_eth_received": 0.0,
            "net_eth": 0.0,
            "total_gas_spent": 0.0,
            "current_eth_balance": current_balance,
        }
    else:
        first_d, last_d = first_last_activity_dates(tx_df)

        row = {
            "wallet": wallet,
            "first_tx_date": first_d,
            "last_tx_date": last_d,
            "tx_count": len(tx_df),
            "total_eth_sent": total_eth_sent(tx_df, address),
            "total_eth_received": total_eth_received(tx_df, address),
            "net_eth": net_eth_from_txs(tx_df, address),
            "total_gas_spent": total_gas_spent_eth(tx_df),
            "current_eth_balance": current_balance,
        }

    summary_df = pd.DataFrame([row])
    return summary_df


def get_wallet_activity_table(address, start_date=None, end_date=None):
    """
    Build wallet_activity table with daily aggregates. For Tableau.

    Columns: wallet, period (date), tx_count, eth_sent, eth_received, gas_spent.

    - start_date / end_date: optional date-like (str YYYY-MM-DD or date). If both None, returns daily activity for full history (default).
    - If provided, only transactions within [start_date, end_date] are included, still aggregated by day.
    """
    try:
        get_transactions
    except NameError:
        raise NameError(
            "get_transactions is not defined. Run the Phase 1 cell "
            "'Transactions and token_transfers in project schema' first."
        )
    tx_df = get_transactions(address)
    wallet = address

    if tx_df.empty:
        out = pd.DataFrame(columns=["wallet", "period", "tx_count", "eth_sent", "eth_received", "gas_spent"])
        return out

    tx_df = tx_df.copy()
    tx_df["_date"] = tx_df["timestamp"].apply(_parse_ts)
    tx_df = tx_df.dropna(subset=["_date"])

    if start_date is not None or end_date is not None:
        if isinstance(start_date, str):
            start_date = datetime.strptime(start_date[:10], "%Y-%m-%d").date()
        if isinstance(end_date, str):
            end_date = datetime.strptime(end_date[:10], "%Y-%m-%d").date()
        if start_date is not None:
            tx_df = tx_df[tx_df["_date"] >= start_date]
        if end_date is not None:
            tx_df = tx_df[tx_df["_date"] <= end_date]

    if tx_df.empty:
        out = pd.DataFrame(columns=["wallet", "period", "tx_count", "eth_sent", "eth_received", "gas_spent"])
        return out

    tx_df["_gas_eth"] = tx_df.apply(_gas_spent_eth, axis=1)
    addr_lower = address.lower()

    def eth_sent_for_group(g):
        return g.loc[g["from_address"].str.lower() == addr_lower, "value_eth"].sum()

    def eth_received_for_group(g):
        return g.loc[g["to_address"].str.lower() == addr_lower, "value_eth"].sum()

    by_date = tx_df.groupby("_date", as_index=False).agg(
        tx_count=("tx_hash", "count"),
        gas_spent=("_gas_eth", "sum"),
    )
    sent_by_date = tx_df.groupby("_date").apply(eth_sent_for_group)
    received_by_date = tx_df.groupby("_date").apply(eth_received_for_group)
    by_date["eth_sent"] = by_date["_date"].map(sent_by_date).fillna(0)
    by_date["eth_received"] = by_date["_date"].map(received_by_date).fillna(0)
    by_date["wallet"] = wallet
    by_date = by_date.rename(columns={"_date": "period"})
    by_date = by_date[["wallet", "period", "tx_count", "eth_sent", "eth_received", "gas_spent"]]

    return by_date



In [11]:
#Export to CSV

def export_to_csv(summary_df, activity_df, token_df):
    """Export wallet_summary, wallet_activity and token_transfers DataFrames to CSV for Tableau Public."""
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    date_str = datetime.now().strftime("%Y-%m-%d")

    summary_path = os.path.join(OUTPUT_DIR, f"wallet_summary_{date_str}.csv")
    activity_path = os.path.join(OUTPUT_DIR, f"wallet_activity_{date_str}.csv")
    token_path = os.path.join(OUTPUT_DIR, f"token_transfers_{date_str}.csv")

    summary_df.to_csv(summary_path, index=False)
    activity_df.to_csv(activity_path, index=False)
    token_df.to_csv(token_path, index=False)

    print(f"Exported summary to {summary_path}")
    print(f"Exported activity to {activity_path}")
    print(f"Exported token transfers to {token_path}")

In [ ]:
#Save_to_db() and run_pipeline()

def save_to_db(summary_df, activity_df, token_df):
    """Save wallet_summary, wallet_activity and token_transfers DataFrames to PostgreSQL."""
    conn = get_db_connection()
    cursor = conn.cursor()

    wallet = summary_df["wallet"].iloc[0]
    cursor.execute("DELETE FROM wallet_summary WHERE wallet = %s", (wallet,))
    cursor.execute("DELETE FROM wallet_activity WHERE wallet = %s", (wallet,))
    cursor.execute("DELETE FROM token_transfers WHERE from_address = %s OR to_address = %s", (wallet, wallet))

    # Insert summary row
    for _, row in summary_df.iterrows():
        cursor.execute("""
            INSERT INTO wallet_summary (
                wallet, first_tx_date, last_tx_date, tx_count,
                total_eth_sent, total_eth_received, net_eth,
                total_gas_spent, current_eth_balance
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row["wallet"],
            row["first_tx_date"],
            row["last_tx_date"],
            int(row["tx_count"]),
            float(row["total_eth_sent"]),
            float(row["total_eth_received"]),
            float(row["net_eth"]),
            float(row["total_gas_spent"]),
            float(row["current_eth_balance"]),
        ))

    # Insert activity rows
    for _, row in activity_df.iterrows():
        cursor.execute("""
            INSERT INTO wallet_activity (
                wallet, period, tx_count, eth_sent, eth_received, gas_spent
            ) VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            row["wallet"],
            row["period"],
            int(row["tx_count"]),
            float(row["eth_sent"]),
            float(row["eth_received"]),
            float(row["gas_spent"]),
        ))

    # Insert token transfer rows
    for _, row in token_df.iterrows():
        cursor.execute("""
            INSERT INTO token_transfers (
                tx_hash, token_symbol, token_contract,
                from_address, to_address, value, timestamp
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row["tx_hash"],
            row["token_symbol"],
            row["token_contract"],
            row["from_address"],
            row["to_address"],
            float(row["value"]) if row["value"] is not None else None,
            row["timestamp"],
        ))

    conn.commit()
    cursor.close()
    conn.close()
    print(f"Saved data for wallet {wallet} to PostgreSQL")



def run_pipeline(address):
    """Fetch wallet data from Alchemy, save to PostgreSQL and export CSVs for Tableau Public."""
    print(f"Running pipeline for {address}...")

    summary = get_wallet_summary_table(address)
    activity = get_wallet_activity_table(address)
    token_transfers = get_token_transfers(address)
    save_to_db(summary, activity, token_transfers)
    export_to_csv(summary, activity, token_transfers)

    print("Pipeline complete.")

# Run it
run_pipeline("0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045")

Running pipeline for 0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045...


/var/folders/bv/b5k07hp50fnfylh6_nb6v9zr0000gn/T/ipykernel_18382/3391460142.py:182: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sent_by_date = tx_df.groupby("_date").apply(eth_sent_for_group)
/var/folders/bv/b5k07hp50fnfylh6_nb6v9zr0000gn/T/ipykernel_18382/3391460142.py:183: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  received_by_date = tx_df.groupby("_date").apply(eth_received_for_group)


SyntaxError: syntax error at or near "h"
LINE 5:                 h,
                        ^


In [14]:
test = get_token_transfers("0xd8dA6BF26964aF9D7eEd9e03E53415D37aA96045")
print(test[test["token_symbol"] == "NOT"][["token_symbol", "token_contract", "value"]])

   token_symbol                              token_contract         value
12          NOT  0x0027449bf0887ca3e431d263ffdefb244d95b555  1.157921e+41
13          NOT  0x0027449bf0887ca3e431d263ffdefb244d95b555  1.157921e+41
